<a href="https://colab.research.google.com/github/rajvirvyas/Senior-Project/blob/main/Image_Processing_CNN_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN approach using Kaggle Dataset**

First, I will need to use data generators to load images from the directories for training, testing and validation.

In [9]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGLE_USR"] = userdata.get('KAGGLE_USR')

In [11]:
# !kaggle datasets download -d marcozuppelli/stegoimagesdataset

! unzip "stegoimagesdataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: val/val/stego/image_06334_ps_0.png  
  inflating: val/val/stego/image_06334_ps_1.png  
  inflating: val/val/stego/image_06335_eth_0.png  
  inflating: val/val/stego/image_06335_html_0.png  
  inflating: val/val/stego/image_06335_url_0.png  
  inflating: val/val/stego/image_06336_html_0.png  
  inflating: val/val/stego/image_06336_html_1.png  
  inflating: val/val/stego/image_06336_url_0.png  
  inflating: val/val/stego/image_06337_html_0.png  
  inflating: val/val/stego/image_06337_js_0.png  
  inflating: val/val/stego/image_06337_url_0.png  
  inflating: val/val/stego/image_06338_eth_0.png  
  inflating: val/val/stego/image_06338_js_0.png  
  inflating: val/val/stego/image_06338_url_0.png  
  inflating: val/val/stego/image_06339_eth_0.png  
  inflating: val/val/stego/image_06339_html_0.png  
  inflating: val/val/stego/image_06339_js_0.png  
  inflating: val/val/stego/image_06340_html_0.png  
  inflating: val/val/stego/ima

In [14]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale= 1./255)  # applied normalization of pixel values
train_generator = train_datagen.flow_from_directory(
    '/content/train/train', #training directory
    target_size= (128, 128),
    batch_size= 32,
    class_mode= 'binary'
)

validation_datagen = ImageDataGenerator(rescale= 1./255)  # applied normalization of pixel values
validation_generator = validation_datagen.flow_from_directory(
    '/content/val', #validation directory
    target_size= (128, 128),
    batch_size= 32,
    class_mode= 'binary'
)

# testing_datagen = ImageDataGenerator(rescale= 1./255)  # applied normalization of pixel values
# testing_generator = testing_datagen.flow_from_directory(
#     '', #training directory
#     target_size= (128, 128),
#     batch_size= 32,
#     class_mode= 'binary'
# )



Found 16000 images belonging to 2 classes.
Found 8000 images belonging to 1 classes.


In [15]:
from keras import layers,models

def CNN_algorithm(shape):
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=shape))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  #model.summary()
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

model=CNN_algorithm((128,128,3))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 14, 14, 128)      

In [17]:
history =model.fit(
    train_generator,
    epochs=10,
    validation_data = validation_generator
)

Epoch 1/10
500/500 [==============================] - 108s 216ms/step - loss: 0.5667 - accuracy: 0.7500 - val_loss: 1.1515 - val_accuracy: 0.0000e+00
Epoch 2/10
500/500 [==============================] - 104s 208ms/step - loss: 0.5647 - accuracy: 0.7500 - val_loss: 1.3434 - val_accuracy: 0.0000e+00
Epoch 3/10
500/500 [==============================] - 105s 211ms/step - loss: 0.5634 - accuracy: 0.7500 - val_loss: 1.2415 - val_accuracy: 0.0000e+00
Epoch 4/10
500/500 [==============================] - 106s 212ms/step - loss: 0.5635 - accuracy: 0.7500 - val_loss: 1.4068 - val_accuracy: 0.0000e+00
Epoch 5/10
500/500 [==============================] - 107s 215ms/step - loss: 0.5636 - accuracy: 0.7500 - val_loss: 1.3075 - val_accuracy: 0.0000e+00
Epoch 6/10
416/500 [=======================>......] - ETA: 12s - loss: 0.5649 - accuracy: 0.7492

KeyboardInterrupt: 

I observed that accuracy doesn't really change and my val_accuracy stays 0. I think it might be worth looking into a different number of layers, or different optimizer or learning rate as well. Also, maybe I need to shuffle the dataset?

In [20]:


def count_images(directory):
    count = 0
    for subdir, _, files in os.walk(directory):
        count += len(files)
    return count

print("Training cover images:", count_images(os.path.join('/content/train/train', 'clean')))
print("Training stego images:", count_images(os.path.join('/content/train/train', 'stego')))
#print("Validation cover images:", count_images(os.path.join(val_dir, 'cover')))
#print("Validation stego images:", count_images(os.path.join(val_dir, 'stego')))


Training cover images: 4000
Training stego images: 12000


There is an issue of data imbalance: 4000 cover images for 12000 stego images;
This can cause my model to work poorly due to biased training.

Before addressing this issue, I need to address that the benchmark paper also utilized a deep learning approach using neural networks in order to detect stego, but their model has some finetuning I must be missing since they got about 99.2% accuracy.